# Introduction

## The Silhouettes Dataset

It's not yet possible to "try before you buy" online so most major clothing retailers include images of models in their items on their ecommerce sites to give shoppers an idea of what to expect. However, popular online second-hand markets (ie. Poshmark, Depop, EBay, Grailed, Vinted... etc.) are comprised of individuals reselling their used garments or rare specially sourced vintage or high fashion items. These individual sellers are not always able to model their listings and are not obligated to find models that can. As a result, it can be challenging for fashion-savvy consumers browsing these online markets to identify items in the styles and silhouettes that they seek. 

This dataset was designed to study 3 popular silhouettes of women's garment tops: triangle/trapeze, hourglass, and tubular-- particularly when the garment is *not* being worn on a body. The silhouette is the shape that a garment takes when worn on the body. Silhouettes are essential to personal style as different silhouettes accentuate the body in different ways. Silhouettes are created by the cut of the garment, the material used to construct the garment, and by the physique of the individual wearing the garment. Since clothing models are not always feasible for small sellers, prospective buyers would benefit from understanding the unique features of different silhouettes when they are not being worn.  

The dataset consists of 1305 35x35 images of clothing items belonging to 3 classes: 'babydoll_trapeze', 'bustier_hourglass', and 'tshirt_straight', each corresponding to a common silhouette. The 'babydoll_trapeze' class represents the triangle/trapeze silhouette. Garments that feature the triangle/trapeze silhouette have a high waist, set just under the bust and a flowing, loose bodice that flares outwards, creating a trapezoidal shape. The silhouette is created by the shape of the bodice, so the type of neckline, sleeve length, and overall length of a garment does not matter. The 'bustier_hourglass' class represents the hourglass silhouette which features a bodice with with exagerated waist indentions that accentuate the bust and the hips. If the length of the bodice extends down to the hips (ie. full length instead of cropped) then the shoulder and hip width should be nearly equal, thus creating an hourglass shape. When not worn on the body, this silhouette is ideally represented by bustier or corset tops. Since corsets and bustiers usually feature 'boning' or are made of stiff materials to cinch the waistline in an aggressive manner, they still retain an hourglass shape when not worn. The 'tshirt_straight' class represents the rectangular or tubular silhouette. As can be inferred from the name, the bodice of a tubular or rectangular garment falls in a straight line, perpendicular to the top of the shoulder and the bottom hem. This silhouette can be represented by any loose-cut t-shirt or shirt. 

 Each class contains 435 hand-selected images from the image gallery, Pinterest and bulk downloaded with *gallery-dl*. Each selection represents an unique garment from the target class either lying flat or on a hanger. The majority of the selected images were not professionally taken and were posted on Pinterest by resellers/sellers to represent their item listings on resale markets. Selecting these types of images preserves some of the variance in angles, exposures, and photo perspectives that an actual consumer would encounter when browsing listings from different sellers. The downloaded images were additionally preprocessed with *rembg* to remove noise contributed by backgrounds and *tensorflow* to remove color channels from the images, create class labels from subdirectory folder names, as well as uniformly resize all images to 35x35 pixels. The decision to remove backgrounds from the images stemmed from the reality that many of the images were not professionally taken. Images of garments were taken on top of brightly patterned bedding, grass, carpets, etc. that could create unwanted distractions from the focal point of the image. At the same time, removing backgrounds does not change the variations in lighting, angles, and photo perspectives of the actual items in the images. The images were transformed to grayscale because a garment's color or pattern does not define it's silhouette. The figure below shows a side by side comparison of an image from the hourglass silhouette class before and after background removal.


 Processing data in *tensorflow* yields a *tf.data.Dataset* object of a tuple (images, labels) where images is of shape (batch_size, image_size[0], image_size[1], num_channels). This dataset was processed in batches of 35 in grayscale (num_channels=1)so images took the shape (35, 35, 35, 1). The labels are integer values of 0, 1, 2 corresponding to each of the three classes so labels are an int32 tensor of shape (35, ). In order to use the dataset outside of *tensorflow*, *tf.data.Dataset* object was unbatched using the function *.unbatch()* and mapped as a list using a lambda function into a (1305, 1225) array of post-processed 35x35 images and into a (1305, ) vector of class labels. The post-processed output is saved as binary numpy arrays 'images.npy' and 'labels.npy' that can be found at: https://github.com/uysiri/604-finalproject 

## Dimensionality Reduction Methods

Dimensionality reduction methods assign instances to real-value vectors in a lower dimensional spaces while preserving the relationships between the instances. The objective of applying these transformations is to reduce noise in datasets. The three dimensionality reduction methods tested on this dataset are: Principal Component Analysis (PCA), Kernel Principal Component Analysis (KPCA), and Isomap. 

### PCA Overview

PCA is a linear method for an orthogonal basis transformation. For a dataset, $x_k, k=1,...l, x_k \in R^N$, PCA is a basis transformation to diagonalize the estimated covariance matrix of the data defined as $COV = \frac{1}{l} \sum_{j=1}^{l} x_jx_j^T$. The *principal componenents* are the orthogonal projections onto the eigenvectors-- a linear combination of the original variables and the elements of the eigenvectors of the covariance matrix. For this project, the library sci-kit learn is used to apply PCA by computing the singular value decomposition (SVD) of the dataset, defined as $X = USV^T$. The principal axes are contained in the columns of $V$ and the columns of $U$ contain the scaled principal components. $S$ is a diagonal matrix of the singular values. The principal axes correspond with the eigenvectors of the covariance matrix whose eigenvalues are $\lambda = \frac{S^2}{l}$. 

### Kernal PCA Overview

The objective of Kernel PCA is to understand non-linear relationships in a dataset. KPCA overcomes the linear limitations of PCA by using a *kernel trick* that maps data into a higher-dimensional feature space before commencing PCA. Given dataset $x_n, n=1,...l, x_n \in R^N$, KPCA has two parts: transforming the data nonlinearly into the feature space $F$ by $\phi : R^N \rightarrow F, x \mapsto X$ then solves the linear PCA for the feature vectors (solve for the directions in $\phi$ that maximizes the variance of $\phi(x)$). 

The algorithm to compute Kernel PCs:

1) Select a kernel function $k(x,y)$ 
    - The inner product of two feature vectors is $\langle \phi(x_i),\phi(x_j) \rangle_F = k(x_i, x_j)$. This is the *kernel trick* where the kernel can evaluate the inner product of any two feature vectors efficiently without knowing $\phi(.)$ or $F$.
    - Gaussian Radial Basis Function (RBF): $k(x,y) = exp(\frac{-||x-y||^2}{2\theta^2})$
    - Polynomial: $k(x,y) = (x * y)^d$
    - Sigmoid: $k(x,y) = \tanh(k(x * y) + \Theta)$

2) Compute a Gram (Kernel) matrix $K_{ij} = k(\phi(x_i), \phi(x_j))$
    - The non-linear feature map is determined by a postive definite kernel-- which is a symmetric kernel $k(.,.)$ defined on a space $F$ where the Gram matrix $(k(x_i,x_j))_{ij}$ is postive semi-definite for an arbitrary number of points $x_1,.., x_n \in x$.
    - The Gram matrix $G = US^2U^T$ has eigenvectors $U$ and eigenvalues $S^2$

3) Center the Kernel matrix
    - $K_{centered} = K - 1_nK - K1_n + 1_nK1_n$ where $1_n$ is a $n x n$ matrix with elements equal to $\frac{1}{n}$

4) Find $U$ and $S^2$ of $K_{centered}$ and multiply each eigenvector by the square root of the corresponding eigenvalue 

### t-SNE Overview

t-SNE is a non-linear method that transforms high dimensional data points to joint probabilities and uses these probabilities to minimize the Kullback-Leibler divergence to obtain low dimensional embeddings. The Kullback-Leibler divergence is the statistical distance where it represents the calculation of how one probability distribution is different from the other one

The t-SNE algorithm:

1)  Calculate the joint probabilities between the data points that represent the similarity between points

2)  Assigns the similarity between the data points on the basis of the calculated joint probability

3)  Represent the data points on lower dimensions on the basis of probability distribution until the minimum Kullback-Leibler divergence 

## Classification Methods 

The classification algorithms selected for this project are: K-Nearest Neighbors (KNN) and  Support Vector Machines (SVM). While Convolutional Neural Networks (CNNs) are highly effective at learning spatial and temporal relationships between pixels in images and would have been excellent for classifying garment silhouettes, the dataset is too small to be used to train neural networks at this time. Each selected classification method will be fit and evaluated for classification performance on the raw data (d=1225) which retains all 1225 image features, the data transformed by PCA, the data trasnformed by KPCA, and the data trasnformed by t-SNE. 

Two classification methods have been selected in order to additionally compare the performances of different algorithms on different data representation methods. The Sci-kit library will be used for all classification tasks in this project.

### K-Nearest Neighbors Overview

The K-Nearest Neighbors (KNN) algorithm is a non-parametric, lazy, supervised learning algorithm that uses the computed distance between instances to assign instances to classes based on their proximity. KNN is non-parametric because it does not make assumptions about the underlying distribution of the data. This algorithm is considered 'lazy' because it relies on memorizing the training set rather than learning a discriminative function. The classification mechanism of this algorithm relies on the assumption that similar input instances have similar outputs. The KNN algorithm requires some designated number of neighbors $K$ to define the neighborhood around an instance. A distance metric (ie. Euclidean, Manhattan, etc.) will be used to calculate the proximity of the $K$ nearest neighbors of the instance being classified. The instance will then be assigned a class based on a majority vote of nearest neighborhors. 

Example Distance Metrics:
- Euclidean: $d(x ,\tilde{x}) = \sqrt{\sum_{j=1}^{l} (x_j - \tilde{x_j})^2}$
    - Based on the Pythagorean theorem, for each dimension $l$ subtract one instance $x_j$ from the other to get the length of the hypotneuse of the triangle between them and square it. The square root of the summation of squared lengths is the Euclidean distance. 
- Manhattan: $d(x, \tilde{x}) = \sum_{j=1}^{l}| x_j - \tilde{x_j} |$
    - The Manhattan distance in an $l$-dimensional space is the cumulative sum of the absolute value distance between $x_j$ and neighboring points at each step. 

A major drawback of the KNN classifier is the curse of dimensionality. In high-dimensional spaces, training points randomly drawn from the dataset may not be close together in space anymore. As a result, the number of training samples needed grows exponentially with dimensionality. Fortunately, this can be generally overcome by the dimensionality reduction methods described above.

### Support Vector Machines Overview 

Support Vector Machines (SVM) is an eager, supervised learning algorithm that aims to find some hyperplane in an $l$-dimensional space that can separate the data points into respective classes. Unlike 'lazy' learners, 'eager' learners apply some discriminative function learned from the training data to classify the test data. To separate classes of instances, the objective of SVM is to find a hyperplane that maximizes the margin/distance between two classes. Hyperplanes can be likened to a city street, where the street should be as wide as possible without encroaching on the sidewalk where people stand. Support vectors are the points that are close to the hyperplane or fall into the 'gutter' of the boundary and direct the position and orientation of the hyperplane. The decision boundary can be linear or non-linear. For this project, non-linear SVM is used.  

The equation that defines the linear hyperplane is $w^Tx + b = 0$ where $w$ is a weight vector, $x$ is an input vector and $b$ is bias. The distance between the sides of the boundary is equal to $\frac{2}{||w||}$, so $||w||$ must be minimized to maximize the margin of the plane. The primal form of the optimization problem can be written in Lagrangian form as: $minL_p = \frac{1}{2}||w||^2 - \sum_{i=1}^{n} a_iy_i(x_iw_i + b) + \sum_{i=1}^{n} a_i$ s.t. $a_i \geq 0$ where $n$ is the number of training points. $w = \sum_{i=1}^{n} a_iy_ix_i$ can be substituted into the primal form for $w$ and $\sum_{i=1}^{n} a_iy_i = 0$ can be subsituted for $b$ to achieve the dual problem $maxL_D(a_i) = \sum_{i=1}^{n} a_i - \frac{1}{2}\sum_{i=1}^{n} a_ia_jy_iy_j(x_i, x_j)$ s.t. $\sum_{i=1}^{n} a_iy_i = 0, a_i \geq 0$. By taking the derivative wrt to $a$ and setting it to 0, we can solve for $a_i$ and find the weights for the maximal margin. 

The dual problem for the linear hyperplane contains the dot products of $x_i$ and $x_j$. This indicates that it is possible to utilize a *kernel trick* to gain linearly separation by mapping the data to a higher dimensional space. The *kernel trick* changes the optimization problem to $L = \sum a_i - \frac{1}{2}\sum a_ia_jy_iy_j K(x_i, x_j)$ where $K(x_i, x_j)$ is the kernel of choice.




## Raw Data Classification

The raw data consists of 1305 35x35 images that is flattened into a (1305, 1225) array. Each image has 1225 dimensions so the Silhouettes dataset can be considered highly dimensional. The high-dimensionality of the dataset  with respect to the relatively lightweight size of samples (1305) can potentially hinder classification tasks for both the KNN and SVM classifier. However, we can observe from the first image of each class that each class is visually distinct from the other.

### Optimizing the Classification Models

In order to identify the optimal hyper-parameters for the KNN and SVM classifiers, we use GridSearchCV to test the performance of potential combinations of hyper-parameters on a sample of the dataset with cross validation. Due to the small size of the dataset, the dataset was split in a stratified manner to preserve class sizes into a training set of 1109 images and a testing set of 196 images. The hyper-parameter combinations were then tested on the training set of raw data images.

The most important hyper-parameter for the KNN classifier is the $K$ number of neighbors to consider. For each $K = 1,...,49$, GridSearchCV initiates and fits a new KNN model with euclidean distance parameters and given $K$ on the training set accross 5 cross validated iterations, totalling 245 iterations. Using this method, we identified the optimal hyper-paramter is $K = 3$ neighbors for our dataset with an average global accuracy score of 83.68% from 5 cross-validation rounds. The effects of hyper-parameter testing for KNN is relatively simple and easily visualized.

Comparatively, SVM has far more hyper-parameters to tune than KNN. In order to achieve an optimal model, hyper-parameters for $C$ the regularization parameter or penalty parameter, $gamma$ defines how far the influence of a single training example reaches, and the $kernel$ to use to non-linearly map the data for classification. The parameters tested for $C$:[0.1,1,10,100], $gamma$:[0.0001,0.001,0.1,1], and $kernel$: ['rbf', 'poly']. For larger values of $C$ the optimization will opt for a narrow-margin hyperplane, increasing the penalty for misclassification while smaller values do the opposite. If $gamma$ is too large, the radius of the area of influence of the support vectors only includes the support vectors, but if gamma is too small the model will fail to capture the complexity of the data. The ideal kernel depends on the underlying dataset itself so the best way to choose is through testing different ones. Using GridSearchCV and the designated training set, the SVM model was initiated with 5 cross validated folds for the 32 possible hyper-parameter combinations, totalling 160 fits. The optimal hyper-parameters obtained from this test were: {'C': 0.1, 'gamma': 1, 'kernel': 'poly'}. The optimal mean score was 89.18% for 5 folds. The top results from hyper-parameter testing for SVM have been compiled into a table because the drastic variations in scores from different combinations of hyper-parameters would make a graph extremely difficult to read.

### Classifying 'Raw' Data

Having replicated the optimal hyper-parameters for the KNN model and for the SVM model, we can now define the KNN and SVM models we are using with the new hyper-parameters. Both the KNN and SVM models are fit on the training dataset of raw data images that was also previously used to identify the optimal hyper-parameters. The training instance contains 1109 images sampled randomly from the dataset. Both of the models are then evaluated on their respective performances in determining the class labels of the test set of 196 images. 

The overal accuracy score for the KNN model was 84% compared to 92% from the SVM. 

We can observe from the confusion matrices that the KNN model and SVM differ in their classification performances across the different classes. While both models struggled to classify the 'bustier/hourglass' class, the KNN model fared a bit worse-- with 25 false negatives and 5 false positives compared to 9 and 2 from the SVM. This is can be hypothetically attributed to the previously mentioned fact that the 'bustier/hourglass' silhouette does not usually take on a strong hourglass shape until it is on the body. Though we have attempted to overcome this by selecting types of clothing made of stiff materials that hold their shape to represent the class, there are likely still instances of 'bustier/hourglass' images that resemble the silhouettes of the 't-shirt/straight' or 'babydoll/trapeze' classes. 

## PCA Data Representation

By applying PCA to the Silhouettes dataset, we hope to reduce some of the noise and complexity that may be leading to misclassification instances. Applying PCA ideally reduces the overall dimensionality while still preserving most of the variance that exists in the data. The question to answer is: What is the fewest number of dimensions that can explain the majority of the variance?

We can observe from the scree plot that just a handful of PC components is able to explain nearly 70% of the datasets variance. This also suggests that the relationships in the data can be linearly explained and that PCA is a good method for reducing the dimensionality of this dataset. 

In order to test for the optimal number of principal components (features/dimensions) to retain in the dataset for classification, we tested potential number of components as the percentage of variance we want the dataset to retain after PCA: [0.7,0.72,0.74,0.76,0.78,0.8,0.82,0.84,0.86,0.88,0.9,0.92,0.94,0.96]. Sci-kit's PCA function is particularly helpful as it takes float values of explained variance as the argument for the number of principal components. For each value of explained variance, the *unsplit* images dataset is fit and reduced to obtain the corresponding coefficients and scores. Since PCA is a linear method, it is reasonable to reconstruct the original images in the new PCA basis (dot product of score and coefficients) to use for classification. For each PCA instance, the new basis images are split into the same sample training set and testing set as the raw data method (by preserving the random state we can replicate the split over and over). The optimal KNN and SVM models are then trained on the transformed training set and evaluated on the transformed testing set for each PCA instance. The scores from each iteration for each model have been recorded and displayed below. 

### Classifying the PCA Basis

We have observed from the graph that for both the KNN and the SVM model, the ideal amount of variance to preserve in the dataset to maximize classification performance is around 74-76%. While the SVM model performs best at 74% while the KNN model performs best at 76%, the difference in accuracy between the two instances is extremely minute. 

Ultimately, 76% of variance was selected to be retained in the dataset because it emphasizes the massive performance boost in the KNN classifier. Just 27 principal components are needed to retain 76% of the variance in the Silhouettes dataset. This is a massive reduction from the 1225 dimensions of the original dataset. We can observe the first image from each class of the data transformed into a new PCA basis. The images are able to retain their general original form with only 27 principal components. 



This reduction in dimensionality successfully boosted the KNN classifier's performance from 84% to 90%. On the other hand, the SVM's performance did not improve at all. Rather, with 27 principal components and 76% variance retained, the SVM's performance slightly deteriorated from 92% to 91%. It should be noted that if we retained 94% of the variance instead, then the SVM's performance would not be affected at all. This lack of change suggests that the SVM is far less sensitive to outliers and noise in the data than the KNN classifier. 

Between the raw data and the new PCA basis, the misclassification rates for the SVM remained fairly static, which suggests that while linear dimensionality reduction does not necessarily hurt nor help SVM in this specific case. On the other hand, the misclassification rates for the KNN have appeared to improve after the dataset was projected into a new PCA basis. 

{DISCUSS RESULTS FURTHER}

## Kernel PCA Representation

We know that Kernel PCA, the non-linear form of PCA dimensionality reduction has significantly more hyper-parameters to tune in relation to the dataset to transform. Kernel PCA also does not have the attribute of "explained variance" like PCA, eliminating the possibility of using some visuals like a Scree Plot to evaluate the signficance of principal components. Kernel PCA is also different from PCA in the sense that we should not project the transformed data into it's original dimensions to form a new basis for classification. Kernel PCA projects the data into even higher dimensions to determine non-linear relationships in the data. As a result, when the data points are projected back down to their original dimensions, the dataset cannot be reconstructed without error. Therefore, it is more reasonable to utilize transformed the projection of data on the principal components. 

In order to optimally transform the Silhouette dataset for classification, we must evaluate different combinations of hyper-parameters against each of the classification models using GridSearchCV. Like SVM, Kernel PCA also uses kernels to project data into higher dimensions so we must test different $kernels$, values of $gamma$, $degree$ (for polynomial and sigmoid kernel) in addition to adjusting the number of principal components. This is a very computationally expensive process as the set of potential hyper-parameters will be tested for each classification model and cross-validated. Based on the effects of linear PCA on the performances of the KNN model and SVM model, we can hypothesize that Kernel PCA will also affect the two models differently. 

In the interest of reducing runtime, the GridSearch for hyper-parameters is split into multiple batches. The first batch evaluates the performance of n_components: [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500, 600], gamma: np.linspace(0.001, 0.05, 10), and kernel: ["rbf"] on data that is then used to train the optimal KNN model. "np.linspace(0.001, 0.05, 10)" means that the GridSearch will test gamma values from 0.001, increasing by increments of 0.05 until 10. The next batch evaluates the performance of n_components: [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500, 600], gamma: np.linspace(0.001, 0.05, 10), coef0: np.linspace(0.001, 0.05, 10), degree: [1, 2, 3, 4, 5] and kernel: ["poly", "sigmoid"] on data that is then used to train the optimal KNN model. Both batches of potential combinations are cross-validated with 3 folds and the top 5 combinations of hyper-parameters from each batch are displayed below. 

We can see that the overall best hyper-parameters for Kernel PCA to prepare data for the KNN model is where the coef0: 0.028, degree: 2, gamma: 0.0064, kernel: sigmoid, and n_components: 40. This combination achieved the highest average score of 87% compared to all other combinations of hyper-parameters. 

The next two GridSearch batches are used to evaluate the optimal Kernel PCA hyper-parameters in relation to the SVM classifier. Unfortunately, the combination of the two kernel methods only further increases the computational complexity and runtime of testing different Kernel PCA hyper-parameters. 

The first attempt of evaluating Kernel PCA with n_components: [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500, 600], gamma: np.linspace(0.001, 0.05, 10), coef0: np.linspace(0.001, 0.05, 10), degree: [2, 3, 4, 5] and kernel: ["rbf", "sigmoid", "poly"] to prepare data for the SVM classifier with 3 cross validated folds for each combination ran for over 780 minutes (13 hours) without conclusion. This batch ran overnight and ultimately had to be killed. Separating the batch into 3 mini batches by kernel also did not help signficantly as the GridSearch evaluation for just the "poly" kernel ran for over 400 minutes (6+ hours) and ultimately could not be allowed to continue for the sake of time and efficiency. The only way to remedy this issue was to cut down on the number of hyper-parameters being tested. 

The first conclusive batch evaluation for Kernel PCA for the SVM classifier was n_components: [50, 100, 500], gamma: [0.005, 0.01, 0.1, 0.5, 1, 5], degree: [2, 3, 4], kernel: ["poly", "sigmoid"]. For the sake of the runtime, we allowed the coef0 (same as SVM param 'C') to take the default value of 1. This batch ran for nearly an hour before concluding, highlighting how the runtime for the GridSearch increases exponentially with increased hyper-parameters. The second batch repeated the same hyper-parameter for n_components, excluding gamma and degree with the rbf kernel. This computation was the fastest by far and concluded in 29 seconds. The top 5 hyper-parameter combinations from both of these batch evaluations are displayed below. 

From the tables, we can observe that the RBF kernel is likely to be incompatible for separating the points in the Silhouette dataset. For both KNN and SVM, the RBF kernel transformation represented the data poorly in comparison to the Sigmoid and Polynomial kernels. From the tables we can observe that the ideal kernel for representing the data for SVM classification is the polynomial kernel with degree: 4, n_components: 100, gamma: 0.01, yielding an average score of 88%. Since we now know the optimal hyper-parameters, we can replicate the data transformations to achieve the new optimal outcomes in the KNN and SVM classifiers. 

The replication is achieved by initating a new Kernel PCA instance with the optimal hyper-parameters for each classifier. Unlike in PCA, the dataset will be split into training set (1109 images) and testing set prior to being transformed by Kernel PCA. This is because we will not be using the reconstructed images after PCA. The Kernel PCA object will be fit on, and transform the training set, meaning that it will learn the structure of the training set to apply to the test set. The test set will only be transformed into a new PCA basis. The images will only be mapped back into the original 1225 dimensions after classification for the purposes of visualization. The SVM classifier is trained with training data transformed by Kernel PCA with n_components:100, kernel: 'poly', gamma: 0.01, and degree: 4. The KNN classifier is trained with training data transformed by Kernel PCA with n_components: 40, kernel: sigmoid, gamma: 0.0064, coef0: 0.028, and degree: 2. 

After Kernel PCA (n_components: 100), the SVM classifier's score increased by 2% from 91% to 93%. On the other hand, after Kernel PCA (n_components: 40), the overall score of the KNN classifier remained at 90% and did not change at all. The differences in the optimal kernel representations of the same data for these two classification algorithms highlights how they are affected by the underlying structure of the data and seek to understand the data. The KNN model, a lazy learner that memorizes the training data, simply classifies new data points based on their closest neighboring training points. Even though the data is projected into a higher dimensional space by the Sigmoid kernel, the KNN classifier does not make any improvements overall. This is likely because the KNN classifier computes labels by calculating the euclidean distance between a test point and it's nearest neighbors. Whether the points have all been transformed by a kernel function into a higher-dimensional space do not seem to benefit the classifier in this instance. On the other hand, the SVM classifier seemed to benefit from the Kernel transformation. It is interesting that the ideal kernel for this instance of Kernel PCA is the same as the kernel used by the SVM classifier. Hypothetically, it's possible that maintaining this consistency makes the data points more cleanly separable to the SVM as the Kernel PCA has already projected the data points with the same method that the SVM uses the interpret the points. 

## Isomap Representation